[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PiotrTyrakowski/PolishLawLLM-Benchmark/blob/KwiatkowskiML/bielik-notebook/notebooks/bielik.ipynb)

In [ ]:
rm -fr PolishLawLLM-Benchmark/

In [ ]:
!git clone -b KwiatkowskiML/bielik-notebook https://github.com/PiotrTyrakowski/PolishLawLLM-Benchmark.git

In [ ]:
%cd PolishLawLLM-Benchmark/
!pip install -e .

In [ ]:
from google.colab import userdata
from huggingface_hub import login

login(token=userdata.get('HF_TOKEN'))

In [ ]:
from benchmark_framework.models.bielik import BielikModel

bielik = BielikModel("speakleash/Bielik-11B-v2.2-Instruct")

In [ ]:
prompt = "Kto w Polsce jest głównym organem wykonawczym rządu? Odpowiedzi: A) Prezydent RP, B) Sejm, C) Rada Ministrow, D) Trybunal konstytucyjny"

response = bielik.generate_response(prompt)
print(response)